### From Part 1

In [1]:
import time
from math import sqrt
from collections import deque

def lucas():
    yield 2
    a = 2
    b = 1
    while True:
        yield b
        a, b = b, a + b
        
def async_sleep(interval_seconds):
    """
    async_sleep always yields atleast once 
    async_sleep(0) yields exactly once
    """
    start = time.time()
    expiry = start + interval_seconds
    while True:
        yield      
        now = time.time()
        if now >= expiry:
            break

            
def async_is_prime(x):
    if x < 2:
        return False
    for i in range(2, int(sqrt(x)) + 1):
        if x % i == 0:
            return False
        yield from async_sleep(0)
    return True


def async_search(iterable, predicate):
    """
    - async_search is a generator function:
        a) calling async_search always returns a generator object:
        b) search progresses when iterated with next()
        c) Final result "returned" in StopIteration payload
    """
    for item in iterable:
        if predicate(item):
            return item
        yield from async_sleep(0)
    raise ValueError("Not Found")

    
def async_print_matches(iterable, async_predicate):
    for item in iterable:
        matches = yield from async_predicate(item) # allows the predicate to make progress and yield control by invoking with yield from
        if matches:
            print("Found : ", item)
        # yield      # => here we dont need yield
        
        
def async_repetitive_message(message, interval_seconds):
    while True:
        print(message)
        yield from async_sleep(interval_seconds)
        

class Task:
    
    next_id = 0
    
    def __init__(self, routine):
        self.id = Task.next_id
        Task.next_id += 1
        self.routine = routine
        

class Schedular:
    
    def __init__(self):
        self.runnable_tasks = deque()
        self.completed_tasks_results = {}
        self.failed_tasks_errors = {}
        
    def add(self, routine):
        task = Task(routine)
        self.runnable_tasks.append(task)
        return task.id
    
    def run_to_completion(self):
        while len(self.runnable_tasks) != 0:
            task = self.runnable_tasks.popleft()
            print("Running task {}...".format(task.id), end="")
            try:
                yielded = next(task.routine)
            except StopIteration as stopped:
                print("Completed with result: {!r}".format(stopped.value))
                self.completed_tasks_results[task.id] = stopped.value
            except Exception as e:
                print("Failed with exception: {}".format(e))
                self.failed_tasks_errors[task.id] = e
            else:
                assert yielded is None
                print("Now yielded")
                self.runnable_tasks.append(task)

### Refactoring above code to asyncio code

In [2]:
import asyncio
import time
from math import sqrt
from collections import deque


def lucas():
    yield 2
    a = 2
    b = 1
    while True:
        yield b
        a, b = b, a + b
        
            
async def is_prime(x):
    if x < 2:
        return False
    for i in range(2, int(sqrt(x)) + 1):
        if x % i == 0:
            return False
        await asyncio.sleep(0)
    return True


async def search(iterable, predicate):
    """
    - async_search is a generator function:
        a) calling async_search always returns a generator object:
        b) search progresses when iterated with next()
        c) Final result "returned" in StopIteration payload
    """
    for item in iterable:
        if predicate(item):
            return item
        await asyncio.sleep(0)
    raise ValueError("Not Found")


async def print_matches(iterable, async_predicate):
    for item in iterable:
        matches = await async_predicate(item) # allows the predicate to make progress and yield control by invoking with yield from
        if matches:
            print("Found : ", item)
        # yield      # => here we dont need yield
        
        
async def repetitive_message(message, interval_seconds):
    while True:
        print(message)
        await asyncio.sleep(interval_seconds)
        

class Task:
    
    next_id = 0
    
    def __init__(self, routine):
        self.id = Task.next_id
        Task.next_id += 1
        self.routine = routine
        

class Schedular:
    
    def __init__(self):
        self.runnable_tasks = deque()
        self.completed_tasks_results = {}
        self.failed_tasks_errors = {}
        
    def add(self, routine):
        task = Task(routine)
        self.runnable_tasks.append(task)
        return task.id
    
    def run_to_completion(self):
        while len(self.runnable_tasks) != 0:
            task = self.runnable_tasks.popleft()
            print("Running task {}...".format(task.id), end="")
            try:
                yielded = next(task.routine)
            except StopIteration as stopped:
                print("Completed with result: {!r}".format(stopped.value))
                self.completed_tasks_results[task.id] = stopped.value
            except Exception as e:
                print("Failed with exception: {}".format(e))
                self.failed_tasks_errors[task.id] = e
            else:
                assert yielded is None
                print("Now yielded")
                self.runnable_tasks.append(task)

In [3]:
schedular = asyncio.get_event_loop()

In [4]:
# schedular.create_task(repetitive_message("Unattended bagger will be destroyed",2.5))
# schedular.create_task(print_matches(lucas(), is_prime))

# asyncio
- coroutines implements <strong style="color:red">tasks</strong>
- coroutines <strong style="color:green">await</strong> other coroutines
- event-loop schedules <strong style="color:pink">concurrent</strong> tasks
- tasks must <strong style="color:blue">not block</strong>
- awaiting facilitates <strong style="color:lightgreen">context switches</strong>
- to yield control <strong style="color:yellow">without</strong> needing ta result <code>await asyncio.sleep(0)</code>

# Coroutine v/s Coroutine Objects
- try to be precise while naming or documenting

#### 1) Coroutine
- code callable

In [5]:
async def search(iterable, predicate):
    """
    - async_search is a generator function:
        a) calling async_search always returns a generator object:
        b) search progresses when iterated with next()
        c) Final result "returned" in StopIteration payload
    """
    for item in iterable:
        if predicate(item):
            return item
        await asyncio.sleep(0)
    raise ValueError("Not Found")

#### 2) Coroutine Object
- code + execution state = awaitable

In [6]:
c = search(lucas(), is_prime)
c

<coroutine object search at 0x000001A4A6DB84C8>

# Future
- So far to monitor running task, there was no way of monitoring anyway
- We just had to run them and wait for until they are completed
- If there is any dependencies among tasks then we can use Future (i.e. one task waiting for another task to complete)

In [7]:
async def thirteen_digit_prime(x):
    return (await is_prime(x)) and len(str(x)) == 13

In [8]:
async def monitored_search(iterable, predicate, future):
    try:
        found_item = await search(iterable, future)
    except ValueError as not_found:
        future.set_exception(not_found)
    else: # no exception
        future.set_exception(found_item)

In [9]:
async def monitor_future(future, interval_seconds):
    while not future.done():
        print('Waiting...')
        await asyncio.sleep(interval_seconds)
    print('Done')

In [10]:
loop = asyncio.get_event_loop()
future = loop.create_future()

co_obj = monitored_search(lucas(), thirteen_digit_prime, future)

loop.create_task(co_obj)
loop.create_task(monitor_future(future, 1.0))
loop.run_until_complete(future)

print(future.result())
loop.close()

RuntimeError: This event loop is already running

Task exception was never retrieved
future: <Task finished coro=<monitored_search() done, defined at <ipython-input-8-6ecb3640f4cb>:1> exception=TypeError("'_asyncio.Future' object is not callable")>
Traceback (most recent call last):
  File "<ipython-input-8-6ecb3640f4cb>", line 3, in monitored_search
    found_item = await search(iterable, future)
  File "<ipython-input-5-3364f93604ea>", line 9, in search
    if predicate(item):
TypeError: '_asyncio.Future' object is not callable


Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...

- Avoid calling the <code>future = Future()</code> 
- constructor directly - prevents event-loops specializing the future implementation
- That's why we have the <code>future = loop.create_future()</code> factoryfunction

<img src="./task.png"/>

<img src="./task2.png"/>

<img src="./ensure_future.png"/>

<img src="./ensure_future2.png"/>

<img src="./bug_create_task.png"/>

<img src="./gather.png"/>

<img src="./event_loops.png"/>

<img src="./event_loops2.png"/>

<img src="./event_loops_socket.png"/>

<img src="./event_loops_protocol.png"/>